In [8]:
import pandas as pd
import numpy as np
import datetime as dt
from w3lib.html import remove_tags
import re

# Questions
- Check if cleaning the job title only in the main dataframe and after deleting exact duplicates yields more or less duplicates.
- For the duplicates, delete based on which subset? 
    - `subset=['job_title', 'job_description', 'company', 'job_location']`
    - `subset=['job_title', 'company', 'job_location']`
    - `subset=['job_title', 'job_description', 'company', 'job_location', 'post_date']`

In [9]:
from pathlib import Path, PureWindowsPath

main_folder = PureWindowsPath("c:\\Users\\gilnr\\OneDrive - NOVASBE\\Work Project\\Code")
MAIN_FOLDER = Path(main_folder)
DATA_FOLDER = MAIN_FOLDER / "Data"
DATA_FOLDER

WindowsPath('c:/Users/gilnr/OneDrive - NOVASBE/Work Project/Code/Data')

# Load all datasets 

In [10]:
bons_empregos = pd.read_json(DATA_FOLDER / 'bons_empregos_jobs.json')
bons_empregos_2 = pd.read_json(DATA_FOLDER / 'BonsEmpregos.json', lines=True)
career_jet = pd.read_json(DATA_FOLDER / 'career_jet_api.json', lines=True)
carga_de_trabalhos = pd.read_json(DATA_FOLDER / 'CargaDeTrabalhos.json', lines=True)
emprego_xl_2 = pd.read_json(DATA_FOLDER / 'emprego_xl_jobs.json')
emprego_xl = pd.read_json(DATA_FOLDER / 'EmpregoXl.json', lines=True)
emprego_org = pd.read_json(DATA_FOLDER / 'EmpregoOrg.json')
itjobs = pd.read_json(DATA_FOLDER / 'itjobs_api.json', lines=True)
jooble = pd.read_json(DATA_FOLDER / 'jooble_api.json', lines=True)
landing_jobs = pd.read_json(DATA_FOLDER / 'landingjobs_api.json', lines=True)
net_empregos_2 = pd.read_json(DATA_FOLDER / 'net_empregos.json')
net_empregos = pd.read_json(DATA_FOLDER / 'NetEmpregos.json', lines=True)

In [11]:
dataframes = [bons_empregos, bons_empregos_2, career_jet, carga_de_trabalhos, emprego_xl, 
              emprego_xl_2, emprego_org, itjobs, jooble, landing_jobs, net_empregos, net_empregos_2]
total = 0
for i in dataframes:
    print(f'Number of job vacancies: {len(i)}')
    total += len(i)
print(f'Total job vacancies before processing {total}')

Number of job vacancies: 2576
Number of job vacancies: 2427
Number of job vacancies: 39204
Number of job vacancies: 6315
Number of job vacancies: 43276
Number of job vacancies: 36318
Number of job vacancies: 99
Number of job vacancies: 21570
Number of job vacancies: 6428
Number of job vacancies: 3797
Number of job vacancies: 122188
Number of job vacancies: 54800
Total job vacancies before processing 338998


In [12]:
jobs_dfs = [bons_empregos, career_jet, carga_de_trabalhos, emprego_xl, emprego_org, itjobs, jooble, landing_jobs, net_empregos]
websites = ['Bons empregos', 'Career Jet', 'Carga de Trabalhos', 'Emprego XL', 'Emprego.org','ITjobs','Jooble','Landing Jobs','Net-empregos']
total = 0
for idx,i in enumerate(jobs_dfs):
    print(f'Number of job vacancies: {len(i)}, {websites[idx]}')
    total += len(i)
print(f'Total job vacancies before processing {total}')

Number of job vacancies: 2576, Bons empregos
Number of job vacancies: 39204, Career Jet
Number of job vacancies: 6315, Carga de Trabalhos
Number of job vacancies: 43276, Emprego XL
Number of job vacancies: 99, Emprego.org
Number of job vacancies: 21570, ITjobs
Number of job vacancies: 6428, Jooble
Number of job vacancies: 3797, Landing Jobs
Number of job vacancies: 122188, Net-empregos
Total job vacancies before processing 245453


# Data Cleaning
- For each website there are specific categories that we need to attend. Some require filtering for job location, others cleaning the job description, and the post date.

## Drop Job Vacancies
What makes a unique job vacancy?
- For our analysis it will be: [job_title, job_description, company, job_location]

# General Functions

In [13]:
def copy_df(dataframe):
   return dataframe.copy()

def replacenan(dataframe):
    dataframe.replace('nan', np.nan, inplace=True)
    dataframe.replace('', np.nan, inplace=True)
    return dataframe
    
def dropNullJobs(dataframe):
    """
    Drop null values that make an online job vacancy unusable for analysis.
    The subset to drop is: ['post_date', 'job_title', 'job_description']
    """
    dataframe.dropna(subset=['post_date', 'job_title', 'job_description'], inplace=True)
    return dataframe

# remove duplicates
def removeDupes(dataframe, subset=['job_title', 'job_description', 'company', 'job_location']):
    dataframe = dataframe.sort_values(by='post_date').drop_duplicates(subset=subset, keep='last')
    return dataframe

def listToRows(dataframe, column):
    return dataframe.explode(column)

def removeTags(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(remove_tags)
    return dataframe

# Description
def clean_text(text):
    to_replace = ['\r', '\n', '•']
    replace = [' ', ' ', '\n']

    for idx, val in enumerate(to_replace):
        text = text.replace(val, replace[idx])
    text = text.strip()
    return text

def cleanDescription(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(lambda x: clean_text(x))
    return dataframe

def invertDate(x):
    if type(x) == float:
        return np.nan
    date = x.split('-')
    return date[2].strip()+'-'+date[1].strip()+'-'+date[0].strip()

def pipeInvertDate(dataframe, function=invertDate):
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: function(x))
    return dataframe

# Date Related Functions

In [14]:
def postDatePreprocess(dataframe, sep=" "):
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: x.split(sep)[0]) 
    return dataframe

def postDateFillNa(dataframe):
    dataframe['post_date'] = dataframe['post_date'].ffill(limit=1).bfill()
    return dataframe

# convert portuguese months to numbers
def longToShortDate(x, sep):
    months = ['janeiro', 'fevereiro','março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
    months_dic = {value:idx+1 for idx, value in enumerate(months)}
    date = [i.strip() for i in x.split(sep)]
    # return f'{date[0]}/{months_dic[date[1]]}/{date[2]}'
    return f'{date[2]}/{months_dic[date[1]]}/{date[0]}'


# convert to datetime object
def convertToDatetime(dataframe, function, sep=' '):
    # Remove comma from date
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: str(x).lower().replace(',',''))
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: dt.datetime.strptime(function(x, sep), "%Y/%m/%d"))
    return dataframe

# Convert Scrape date to datetime
def toDatetime(dataFrame, columns_list, dayfirst=False):
    for i in columns_list:
        dataFrame[i] = pd.to_datetime(dataFrame[i], dayfirst=dayfirst)
    return dataFrame
    
def notDateToNan(x):
    if re.findall('(0[1-9]|[12][0-9]|3[01])[-](0[1-9]|1[012])[-](19|20)\d\d', str(x)) != []:
        return x
    else:
        return np.nan

def applyFuncToColumn(dataframe, function=notDateToNan, columns_list=['post_date']):
    for i in columns_list:
        dataframe[i] = dataframe[i].apply(lambda x: function(x))
    return dataframe

## Define `Clean Job Title` Functions

In [15]:
import functools
from typing import Callable

ComposableFunction = Callable[[str], str]

def compose(*functions: ComposableFunction) -> ComposableFunction:
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions)

def cleanJobChars(x: str) -> str:
    # Capitalize the job title
    x = x.lower()
    stop_chars = ['(m/f)', 'm/f', '-', ':', ' - ', ' – ', '(remote)', ' / ', '(', ' para ', '_']
    hyphen_exceptions = ['-se', '-o', '-a', '-os', '-as', 'e-', '-e']
    title_position_exceptions = ['(junior)', '(senior)']
    
    def exception_handle(x: str, exception_list: list):
        ex = False
        for exception in exception_list:
            if exception in x:
                ex = True
            if ex == False:
                aux = x.split(stop)
                for val in aux:
                    if val != stop and val != '':
                        x = val
                        break
        return x
    
    for stop in stop_chars:
        if stop in x:
            if stop == '-':
                exception_handle(x, hyphen_exceptions)
            if stop == '(':
                exception_handle(x, title_position_exceptions)
            if stop == '_':
                x = x.split(stop)[0]
            else:  
                aux = x.split(stop)
                for val in aux:
                    if val != stop and val != '':
                        x = val
                        break
    return x.strip()

def replaceGenderWords(x: str) -> str:
    gender_words = ['/a', '/o', '/as', '/os', '/e', '/es']
    for i in gender_words:
        x = x.replace(i, '')
    return x

def replaceCommonFillers(x: str) -> str:
    fillers = ['recruta-se para', 'recruta-se', 'oferta de emprego:', 'oferta:', 'oferta de emprego', 'oferta', 'precisa-se', 
               'precisas-se', 'part-time']
    for i in fillers:
        x = x.replace(i, '')
    return x

cleanJobTitle = compose(replaceCommonFillers, replaceGenderWords, cleanJobChars)

# How to use:
# applyFuncToColumn(bons_empregos, function=cleanJobTitle, columns_list=['job_title'])

## Define `Clean Job Location` Functions

In [16]:
# import functools
# from typing import Callable

# ComposableFunction = Callable[[str], str]

# def compose(*functions: ComposableFunction) -> ComposableFunction:
#     return functools.reduce(lambda f, g: lambda x: g(f(x)), functions)

# def splitLocation(x: str) -> str:
#     # Capitalize the job location
#     stop_chars = [' - ', ' – ', ' / ', '/']
#     for stop in stop_chars:
#         if stop in x:
#             return [i.capitalize() for i in x.split(stop)]
#         else:
#             return x

# def replaceGenderWords(x: str) -> str:
#     gender_words = ['/a', '/o', '/as', '/os', '/e', '/es']
#     for i in gender_words:
#         x = x.replace(i, '')
#     return x

# def replaceCommonFillers(x: str) -> str:
#     fillers = ['recruta-se para', 'recruta-se', 'oferta de emprego:', 'oferta:', 'oferta de emprego', 'oferta', 'precisa-se', 
#                'precisas-se', 'part-time']
#     for i in fillers:
#         x = x.replace(i, '')
#     return x

# cleanJobTitle = compose(replaceCommonFillers, replaceGenderWords, splitLocation)

# How to use:
# applyFuncToColumn(bons_empregos, function=cleanJobTitle, columns_list=['job_title'])


# Total jobs grouped by Year and Month

In [17]:
def totalJobsByYearMonth(dataframe):
    dataframe['post_year'] = dataframe['post_date'].dt.year
    dataframe['post_month'] = dataframe['post_date'].dt.month
    return pd.DataFrame(dataframe.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

## Bons Empregos
- Specific functions:
    - `getPortugalLocation`

In [18]:
def getPortugalLocation(dataframe):
    # Get only job offers in Portugal
    dataframe = dataframe.loc[dataframe['job_location'] != 'Estrangeiro'].copy()
    return dataframe

In [19]:
bons_empregos_clean = (bons_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, columns_list=['scrape_date'], dayfirst=True).
                    pipe(getPortugalLocation).
                    pipe(convertToDatetime, longToShortDate).
                    pipe(removeDupes)
)

print(f'Previous shape: {bons_empregos.shape}\nCurrent shape:{bons_empregos_clean.shape}')
bons_empregos_clean.head()

Previous shape: (2576, 8)
Current shape:(1988, 8)


,job_location,job_category,job_description,job_title,post_date,scrape_date,job_href,company
2575,Porto,Outros,"Palavras-chave: Java, Jenkins, Micro-serviços\...",java developer,2020-10-06,2021-10-05,https://www.bonsempregos.com/oferta-emprego/ja...,Dellent
2569,Porto,Outros,A Keller Williams é uma empresa de consultoria...,gestor,2020-10-06,2021-10-05,https://www.bonsempregos.com/oferta-emprego/ge...,NaN
2570,Porto,Outros,Empresa do ramo da Engenharia Eletrotécnica/Me...,engenheiro mecânico,2020-10-06,2021-10-05,https://www.bonsempregos.com/oferta-emprego/en...,NaN
2574,Porto,Outros,Operador de ETAR (M/F) - Braga\nO grupo dst de...,operador de etar,2020-10-06,2021-10-05,https://www.bonsempregos.com/oferta-emprego/op...,grupo
2567,Braga,Comercial e Serviços,A Remax foi escolhida pela revista Exame a Mel...,consultor imobiliário,2020-10-07,2021-10-05,https://www.bonsempregos.com/oferta-emprego/co...,NaN


In [20]:
totalJobsByYearMonth(bons_empregos_clean)

job_title
post_year post_month           
2021      10                 10
          9                 223
          8                  74
          7                 218
          6                 209
          5                 208
          4                 203
          3                 190
          2                 139
          1                 141
2020      12                 99
          11                135
          10                139

## Career Jet

In [21]:
# convert job location to list
career_jet['job_location'] = career_jet['job_location'].apply(lambda x: x.split(','))

In [22]:
career_jet_clean = (career_jet.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, columns_list=['scrape_date', 'post_date'], dayfirst=True).
                    pipe(listToRows, 'job_location').
                    pipe(removeDupes)
)

print(f'Previous shape: {career_jet.shape}\nCurrent shape:{career_jet_clean.shape}')
career_jet_clean.head()

Previous shape: (39204, 9)
Current shape:(7026, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_href,salary
6336,assistente call center,A Vertente Humana é uma empresa que presta ser...,2021-10-21,2021-10-22,NaN,Viana do Castelo,NaN,http://jobviewtrack.com/pt-pt/job-481b416c5e17...,NaN
6337,chefe de receção vilamoura,Chefe de Receção Vilamoura (m/f) Referencia:...,2021-10-21,2021-10-22,Michael Page,Faro,NaN,http://jobviewtrack.com/pt-pt/job-4c48416e4501...,NaN
6338,eletricista,Realizar a manutenção diária em conformidade c...,2021-10-21,2021-10-22,Talenter,Portugal,NaN,http://jobviewtrack.com/pt-pt/job-481e41684101...,NaN
6339,motorista de pesado c+e,"A Vertente Humana, empresa de Trabalho Temporá...",2021-10-21,2021-10-22,Vertente Humana,Setúbal,NaN,http://jobviewtrack.com/pt-pt/job-494d41604210...,NaN
6341,controller de gestão,Ligamos grandes profissionais a grandes empres...,2021-10-21,2021-10-22,Adecco,Porto,NaN,http://jobviewtrack.com/pt-pt/job-1913416e420a...,NaN


In [23]:
assert career_jet_clean.post_date.dtypes == career_jet_clean.scrape_date.dtypes

In [24]:
totalJobsByYearMonth(career_jet_clean)

job_title
post_year post_month           
2021      11               4184
          10               2842

## Carga de Trabalhos

In [25]:
carga_de_trabalhos_clean = (carga_de_trabalhos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, columns_list=['scrape_date'], dayfirst=True).
                    pipe(convertToDatetime, longToShortDate, '/').
                    pipe(removeDupes)
)

print(f'Previous shape: {carga_de_trabalhos.shape}\nCurrent shape:{carga_de_trabalhos_clean.shape}')
carga_de_trabalhos_clean.head()

Previous shape: (6315, 7)
Current shape:(5340, 7)


,job_description,job_title,post_date,scrape_date,company,job_location,job_href
6314,Somos um projecto editorial e queremos recruta...,procuramos designer gráfico,2020-01-01,2021-11-12,SCM Supply Chain Magazine,Alcochete,https://www.cargadetrabalhos.net/2020/01/01/pr...
6313,Procuramos Designer Gráfico para trabalho de c...,designer gráfico,2020-01-01,2021-11-12,EA Produções,"Lisboa, Benfica",https://www.cargadetrabalhos.net/2020/01/01/de...
6312,Somos um estúdio de Design Full Service em exp...,comercial,2020-01-01,2021-11-12,Raio Criativo,Lisboa,https://www.cargadetrabalhos.net/2020/01/01/co...
6311,"As senior Software Engineer and technologist, ...",lead developer,2020-01-02,2021-11-12,Yomp – Digital Design &amp; Development,Guimarães,https://www.cargadetrabalhos.net/2020/01/02/le...
6310,A Agroop é uma empresa de AgriTech que está a ...,marketeer digital,2020-01-02,2021-11-12,Agroop,Sintra (junto ao Taguspark),https://www.cargadetrabalhos.net/2020/01/02/ma...


In [26]:
totalJobsByYearMonth(carga_de_trabalhos_clean)

job_title
post_year post_month           
2021      11                174
          10                321
          9                 307
          8                 256
          7                 259
          6                 296
          5                 349
          4                 272
          3                 280
          2                 225
          1                 202
2020      12                165
          11                200
          10                219
          9                 172
          8                 188
          7                 189
          6                 159
          5                 153
          4                 114
          3                 204
          2                 288
          1                 348

## Emprego XL

In [27]:
emprego_xl_clean = (emprego_xl.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(applyFuncToColumn).
                    pipe(pipeInvertDate).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, columns_list=['scrape_date', 'post_date'], dayfirst=True).
                    # # pipe(convertToDatetime, longToShortDate, '/').
                    pipe(removeDupes)
)

print(f'Previous shape: {emprego_xl.shape}\nCurrent shape:{emprego_xl_clean.shape}')
emprego_xl_clean.head()

Previous shape: (43276, 7)
Current shape:(24633, 7)


,job_description,job_title,post_date,scrape_date,company,job_href,job_location
39090,"A rede CENTURY 21®, fundada nos EUA em 1971, é...",novos talentos comerciais,2020-11-02,2021-10-26,NaN,https://www.empregoxl.com/emprego/434009/novos...,Lisboa
39087,Ser Gestor (a) Imobiliário (a) é acompanhar fa...,já considerou a actividade imobiliária?,2020-11-02,2021-10-26,Remax Metrópole,https://www.empregoxl.com/emprego/434006/ja-co...,Lisboa
39085,"A RE/MAX Liberty, presente no mercado imobiliá...",admitimos 2 profissionais,2020-11-02,2021-10-26,Remax Liberty,https://www.empregoxl.com/emprego/434000/admit...,Braga
39147,A ManpowerGroup Portugal encontra-se a recruta...,apoio ao cliente 09h,2020-11-02,2021-10-26,ManpowerGroup,https://www.empregoxl.com/emprego/433985/apoio...,Lisboa
39146,DESCRICÃO/FUNCÃOPrecisa-se administrativa com ...,coordenadora loja,2020-11-02,2021-10-26,Re/Max Soul,https://www.empregoxl.com/emprego/433983/coord...,Lisboa


In [28]:
totalJobsByYearMonth(emprego_xl_clean)

job_title
post_year post_month           
2021      11               1012
          10               1879
          9                1810
          8                1641
          7                1704
          6                2259
          5                2351
          4                1933
          3                1826
          2                1350
          1                1753
2020      12               1966
          11               3149

## Emprego.Org

In [29]:
emprego_org_clean = (emprego_org.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(postDatePreprocess, '/').
                    pipe(toDatetime, columns_list=['scrape_date'], dayfirst=True).
                    pipe(toDatetime, ['post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {emprego_org.shape}\nCurrent shape:{emprego_org_clean.shape}')
emprego_org_clean.head()

Previous shape: (99, 8)
Current shape:(23, 8)


,job_title,job_description,post_date,scrape_date,job_location,job_href,salary,company
2,abertura nova filial,Somos uma multinacional de comércio e serviços...,2021-10-25,2021-10-26,Porto Porto,https://empregos.org/view.php?job_id=2796890&t...,12000 EUR,None
84,consultor negócios,Pare de adiar o seu sucesso! Encerre 2021 com ...,2021-10-26,2021-10-26,Porto,https://empregos.org/view.php?job_id=2796747&t...,25.000 Ano EUR,Maxgroup
85,gestor comercial,GESTOR COMERCIAL (M/F)\r\n\r\nO Grupo Century2...,2021-10-26,2021-10-26,Lisboa Lisboa,https://empregos.org/view.php?job_id=2796749&t...,Não especificado,century21
92,reforço de equipa,Somos uma empresa MULTINACIONAL em forte expan...,2021-10-26,2021-10-26,Maia Porto,https://empregos.org/view.php?job_id=2796760&t...,10000 EUR,Sumptuoso Crescer unip. lda
93,diretor comercial,"Oportunidade de integrar um projeto de futuro,...",2021-10-26,2021-10-26,Braga Braga,https://empregos.org/view.php?job_id=2796757&t...,1750 EUR,"Sábia Visão - Mediação Imobiliária, Lda"


In [30]:
totalJobsByYearMonth(emprego_org_clean)

,,job_title
post_year,post_month,
2021,10,23


## ITJOBS

In [31]:
def simplifyDate(x):
    return dt.datetime.strptime(x.split(' ')[0], '%Y-%m-%d')
# simplifyDate('2021-09-17 09:11:28')

In [32]:
itjobs_clean = (itjobs.
                    pipe(copy_df).
                    pipe(listToRows, 'job_location').
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(applyFuncToColumn, function=simplifyDate, columns_list=['post_date']).
                    pipe(toDatetime, columns_list=['scrape_date'], dayfirst=True).
                    pipe(toDatetime, ['post_date']).
                    # pipe(.apply(lambda x: dt.datetime.strftime('%Y-%m-%d'))).
                    pipe(removeDupes)
)

print(f'Previous shape: {itjobs.shape}\nCurrent shape:{itjobs_clean.shape}')
itjobs_clean.head()

Previous shape: (21570, 9)
Current shape:(8337, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_ref,salary
3824,java developer,A DECODE é uma empresa onde podes ser tu mesmo...,2021-09-17,2021-10-22,Decode,Lisboa,NaN,https://www.itjobs.pt/oferta/405464/java-devel...,NaN
3840,tech leader frontend,"We believe transparency, trust and integrity a...",2021-09-17,2021-10-22,KCS IT,Porto,NaN,https://www.itjobs.pt/oferta/405665/tech-leade...,NaN
3838,vuejs developer,VueJS Developer (W/M) - Lisboa Your connection...,2021-09-17,2021-10-22,Aubay,Lisboa,NaN,https://www.itjobs.pt/oferta/405746/vuejs-deve...,NaN
3837,scrum master,A Randstad é a empresa número 1 no setor de re...,2021-09-17,2021-10-22,Randstad Portugal,Aveiro,NaN,https://www.itjobs.pt/oferta/405793/agile-master,NaN
3836,.net fullstack developer,The world is more connected than ever before a...,2021-09-17,2021-10-22,Axians,Porto,NaN,https://www.itjobs.pt/oferta/405850/net-fullst...,NaN


In [33]:
totalJobsByYearMonth(itjobs_clean)

job_title
post_year post_month           
2021      11               2840
          10               3879
          9                1618

## Jooble

In [34]:
jooble_clean = (jooble.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, columns_list=['scrape_date', 'post_date'], dayfirst=True).
                    pipe(removeTags, ['job_title']).
                    pipe(removeDupes)
)

print(f'Previous shape: {jooble.shape}\nCurrent shape:{jooble_clean.shape}')
jooble_clean.head()

Previous shape: (6428, 9)
Current shape:(2933, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_ref,salary
728,operador de caixa,"\r\n\r\n Adecco, empresa multinacional especia...",2021-10-22,2021-10-22,Adecco,Aveiro,NaN,https://pt.jooble.org/desc/940066953020983867?...,NaN
729,assistente dentária,A Experiência que gostávamos que tivesse\n\r\n...,2021-10-22,2021-10-22,OralMED Saúde,Lisboa,NaN,https://pt.jooble.org/desc/-463883459362016540...,NaN
734,administrativa comercial,\r\n A Kelly Services é uma empresa de Gestão ...,2021-10-22,2021-10-22,Kelly Services Portugal,Setúbal,NaN,https://pt.jooble.org/desc/-621383893912223577...,NaN
740,rececionista,"Adecco, empresa multinacional especializada n...",2021-10-22,2021-10-22,Adecco,Ourém,NaN,https://pt.jooble.org/desc/46539090233946467?c...,NaN
743,motorista tvde,Parceiro de TODAS as Plataformas Eletrónicas p...,2021-10-22,2021-10-22,JJ Operações Turísticas,Lisboa,NaN,https://pt.jooble.org/away/-138296353078591006...,NaN


In [35]:
totalJobsByYearMonth(jooble_clean)

job_title
post_year post_month           
2021      11               2216
          10                717

## Landing Jobs IT

In [36]:
landing_jobs_clean = (landing_jobs.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(postDatePreprocess, 'T').
                    pipe(toDatetime, columns_list=['scrape_date'], dayfirst=True).
                    pipe(toDatetime, ['post_date']).
                    # pipe(removeTags, 'job_title').
                    pipe(removeDupes)
)

print(f'Previous shape: {landing_jobs.shape}\nCurrent shape:{landing_jobs_clean.shape}')
landing_jobs_clean.head()

Previous shape: (3797, 9)
Current shape:(664, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_ref,salary
1284,back,"We seek flexible people, open to explore diffe...",2018-11-16,2021-10-30,Sky Technology Centre – Portugal,Lisbon,NaN,https://landing.jobs/at/sky-technology-centre-...,NaN
2588,devops manager,We hope you haveAt least 5 years of proven tra...,2020-03-27,2021-11-12,Blip,Porto,NaN,https://landing.jobs/at/blip/devops-manager,NaN
2589,devops engineer,"We hope you haveLinux systems administration, ...",2020-03-27,2021-11-12,Blip,Porto,NaN,https://landing.jobs/at/blip/devops-engineer-i...,NaN
6,senior back,We hope you haveAt least 6 years working with ...,2020-03-27,2021-10-22,Blip,Porto,NaN,https://landing.jobs/at/blip/senior-back-end-d...,NaN
7,qa engineer,We hope you haveAt least 3 years working as a ...,2020-03-27,2021-10-22,Blip,Porto,NaN,https://landing.jobs/at/blip/qa-engineer-in-po...,NaN


In [37]:
totalJobsByYearMonth(landing_jobs_clean)

job_title
post_year post_month           
2021      11                 47
          10                109
          9                  73
          8                 104
          7                  72
          6                  24
          5                  50
          4                  21
          3                  57
          2                  31
          1                   7
2020      12                 34
          11                  5
          10                  8
          9                   1
          8                   4
          7                   3
          6                   1
          5                   3
          4                   2
          3                   7
2018      11                  1

## Net Empregos

In [38]:
net_empregos_clean = (net_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(pipeInvertDate).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    # two pipes are needed beacause - for some reason, the function was not replacing some words it should
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(cleanDescription, ['job_title']).
                    pipe(toDatetime, columns_list=['scrape_date', 'post_date'], dayfirst=True).
                    pipe(removeDupes)
)

print(f'Previous shape: {net_empregos.shape}\nCurrent shape:{net_empregos_clean.shape}')
net_empregos_clean.head()

Previous shape: (122188, 8)
Current shape:(58609, 8)


,job_description,job_title,post_date,scrape_date,company,job_location,job_category,job_href
44201,A Ankix é uma empresa de competências tecnológ...,iosndroid developer,2021-10-22,2021-10-30,Ankix,Lisboa,Informática ( Programação ),https://www.net-empregos.com/7806923/ios-andro...
42396,A AC Consulting encontra-se a recrutar Emprega...,empregado,2021-10-23,2021-10-30,AC Consulting,Lisboa,Restauração / Bares / Pastelarias,https://www.net-empregos.com/7775646/empregado...
42394,Somos líderes no mercado da entrega de Sushi a...,sushiman,2021-10-23,2021-10-30,Sushi em tua casa - Porto,Porto,Restauração / Bares / Pastelarias,https://www.net-empregos.com/7778008/sushiman-...
42392,"A Vincci Hoteles, é uma cadeia Internacional d...",copeira,2021-10-23,2021-10-30,Vincci Hoteles,Porto,Restauração / Bares / Pastelarias,https://www.net-empregos.com/7776747/copeira-m...
42391,Precisa-se de colaborador para desempenhar fun...,precisa,2021-10-23,2021-10-30,Farmões LDA,Lisboa,Farmácia / Biotecnologia,https://www.net-empregos.com/7777988/precisa-s...


In [39]:
totalJobsByYearMonth(net_empregos_clean)

job_title
post_year post_month           
2021      11              36000
          10              22609

# Add Website Column to all dataframes before concat

In [40]:
jobs_dfs = [bons_empregos_clean, career_jet_clean, carga_de_trabalhos_clean, emprego_xl_clean, emprego_org_clean, itjobs_clean, jooble_clean, landing_jobs_clean, net_empregos_clean]
websites = ['Bons empregos', 'Career Jet', 'Carga de Trabalhos', 'Emprego XL', 'Emprego.org','ITjobs','Jooble','Landing Jobs','Net-empregos']

# Add column with website name
for idx, value in enumerate(jobs_dfs):
    value['website'] = websites[idx]

# Concat All dataframes into one for data Deduplication

In [41]:
neworder = ['job_title','job_description','company','job_location','job_category','salary', 'post_date', 'scrape_date','job_href', 'website']

df = pd.concat([i.reindex(columns=neworder) for i in jobs_dfs])

# Validate that the concatenation is happening properly
assert len(df) == sum(len(i) for i in jobs_dfs)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109553 entries, 2575 to 122187
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   job_title        109553 non-null  object        
 1   job_description  109553 non-null  object        
 2   company          102645 non-null  object        
 3   job_location     108610 non-null  object        
 4   job_category     60597 non-null   object        
 5   salary           1970 non-null    object        
 6   post_date        109553 non-null  datetime64[ns]
 7   scrape_date      109553 non-null  datetime64[ns]
 8   job_href         97619 non-null   object        
 9   website          109553 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 9.2+ MB


## Pipeline

In [43]:
def cleanCompany(dataframe):
    def capitalize(x):
        try:
            return x.capitalize()
        except AttributeError:
            return ''
    dataframe['company'] = dataframe['company'].apply(lambda x: capitalize(x))
    return dataframe

In [44]:
df_clean = (df.
            pipe(copy_df).
            pipe(replacenan).sort_values(by='post_date').
            pipe(postDateFillNa).
            pipe(dropNullJobs).
            pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
            pipe(cleanCompany).
            pipe(cleanDescription, ['job_title', 'job_description']).
            pipe(removeDupes, ['job_title', 'job_description','company', 'job_location'])
)
df_clean.reset_index(drop=True, inplace=True)

print(f'Previous shape: {df.shape}\nCurrent shape:{df_clean.shape}')

Previous shape: (109553, 10)
Current shape:(109465, 10)


In [45]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109465 entries, 0 to 109464
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   job_title        109465 non-null  object        
 1   job_description  109465 non-null  object        
 2   company          109465 non-null  object        
 3   job_location     108530 non-null  object        
 4   job_category     60580 non-null   object        
 5   salary           1970 non-null    object        
 6   post_date        109465 non-null  datetime64[ns]
 7   scrape_date      109465 non-null  datetime64[ns]
 8   job_href         97551 non-null   object        
 9   website          109465 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 8.4+ MB


In [46]:
print(f'There is a total of {len(df_clean)} jobs of which {df_clean.job_title.nunique()} have unique titles')

There is a total of 109465 jobs of which 32415 have unique titles


In [47]:
df_clean.describe()

C:\Users\gilnr\AppData\Local\Temp/ipykernel_19124/2722606686.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_clean.describe()


,job_title,job_description,company,job_location,job_category,salary,post_date,scrape_date,job_href,website
count,109465,109465,109465,108530,60580,1970,109465,109465,97551,109465
unique,32415,96353,18362,1744,67,265,668,9,94079,9
top,precisa,A Multipessoal é uma empresa de referência no ...,,Lisboa,Indústria / Produção,665 - 665,2021-11-17 00:00:00,2021-10-26 00:00:00,http://jobviewtrack.com/pt-pt/job-191241685e10...,Net-empregos
freq,2145,212,6908,44538,5244,492,11326,24523,4,58592
first,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-16 00:00:00,2021-10-05 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-17 00:00:00,2021-11-18 00:00:00,NaN,NaN


In [48]:
df_clean.head()

,job_title,job_description,company,job_location,job_category,salary,post_date,scrape_date,job_href,website
0,back,"We seek flexible people, open to explore diffe...",Sky technology centre – portugal,Lisbon,NaN,NaN,2018-11-16,2021-10-30,NaN,Landing Jobs
1,procuramos designer gráfico,Somos um projecto editorial e queremos recruta...,Scm supply chain magazine,Alcochete,NaN,NaN,2020-01-01,2021-11-12,https://www.cargadetrabalhos.net/2020/01/01/pr...,Carga de Trabalhos
2,designer gráfico,Procuramos Designer Gráfico para trabalho de c...,Ea produções,"Lisboa, Benfica",NaN,NaN,2020-01-01,2021-11-12,https://www.cargadetrabalhos.net/2020/01/01/de...,Carga de Trabalhos
3,comercial,Somos um estúdio de Design Full Service em exp...,Raio criativo,Lisboa,NaN,NaN,2020-01-01,2021-11-12,https://www.cargadetrabalhos.net/2020/01/01/co...,Carga de Trabalhos
4,lead developer,"As senior Software Engineer and technologist, ...",Yomp – digital design &amp; development,Guimarães,NaN,NaN,2020-01-02,2021-11-12,https://www.cargadetrabalhos.net/2020/01/02/le...,Carga de Trabalhos


In [49]:
totalJobsByYearMonth(df_clean)

job_title
post_year post_month           
2021      11              46458
          10              32370
          9                4018
          8                2072
          7                2249
          6                2788
          5                2947
          4                2427
          3                2353
          2                1742
          1                2100
2020      12               2261
          11               3485
          10                365
          9                 172
          8                 192
          7                 190
          6                 158
          5                 156
          4                 115
          3                 210
          2                 288
          1                 348
2018      11                  1

In [50]:
# with open(DATA_FOLDER / 'full_data_clean.json', 'w', encoding='utf-8') as file:
#     df_clean.to_json(file, force_ascii=False, orient='records', date_format='iso', date_unit='s')

In [251]:
test = pd.read_json(DATA_FOLDER / 'full_data_clean.json')

In [252]:
locations = pd.read_excel('freguesias-metadata.xlsx')
locations.drop(columns=['nivel', 'dicofre','brasao'], inplace=True)
# locations['dist_conc_freg'] = locations['distrito'] + ' , ' + locations['concelho'] + ' , ' + locations['freguesia']
locations.head()

,distrito,concelho,freguesia
0,Aveiro,Águeda,Aguada de Cima
1,Aveiro,Águeda,Fermentelos
2,Aveiro,Águeda,Macinhata do Vouga
3,Aveiro,Águeda,Valongo do Vouga
4,Aveiro,Águeda,União das freguesias de Águeda e Borralha


# Generate Exact Match Dictionary

In [253]:
from fold_to_ascii import fold as ascii_fold

def normalizeLocationDict(location: str) -> str:
    lowercased_ascii = ascii_fold(location.lower(), 'REMOVE_ME').replace('REMOVE_ME', '').split(',')[0] # split at , and pick first component
    only_alpha = re.sub(r'[^a-z]', ' ', lowercased_ascii)
    sem_unioes = re.sub(r'uniao d\w+ freguesias d\w+ ', '', only_alpha)
    remove_duplicate_spaces = re.sub(r'\s+', ' ', sem_unioes).strip()
    return remove_duplicate_spaces

locations = pd.read_excel('freguesias-metadata.xlsx')

locations_dict = {}
ambiguos = set()
for loc in locations.itertuples():
    for campo in [loc.freguesia, loc.concelho, loc.distrito]:
        normalizedLocation = normalizeLocationDict(campo)
        if normalizedLocation in locations_dict and locations_dict[normalizedLocation] != loc.distrito.lower():
            ambiguos.add(normalizedLocation)
        else:
            locations_dict[normalizedLocation] = loc.distrito.lower()
        
# print(sorted(set(locations_dict.keys())))

locations_dict[normalizeLocationDict('ereira e lapa')]

'santarém'

In [254]:
'cadaval' in ambiguos

False

In [255]:
import functools
from typing import Callable
from strsimpy.jaro_winkler import JaroWinkler


ComposableFunction = Callable[[str], str]

def compose(*functions: ComposableFunction) -> ComposableFunction:
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions)
    
def normalizeLocation(location:str) -> str:
    if location is None:
        return ''
    lowercased_ascii = ascii_fold(location.lower(), 'REMOVE_ME').replace('REMOVE_ME', '').split(',')
    if len(lowercased_ascii) == 1:
        only_alpha = re.sub(r'[^a-z]', ' ', lowercased_ascii[0])
        remove_duplicate_spaces = re.sub(r'\s+', ' ', only_alpha).strip()
        return remove_duplicate_spaces
    else:
        return [re.sub(r'\s+', ' ', re.sub(r'[^a-z]', ' ', v)).strip() for v in lowercased_ascii]


def commonLocationFillers(x:str) -> str:
    portugal_in_other_words = ['todo o pais','todos o pais', 'qualquer zona',
                               'todas as zonas', 'trabalho de casa', 'qualquer']
    for i in portugal_in_other_words: 
        if i in x:
            x = x.replace(i,'portugal') 
    return x.strip()

def replaceWithEmpty(x:str) -> str:
    to_replace = ['ilha de ', 'zona de ', 'e etc']
    for i in to_replace: 
        if i in x:
            x = x.replace(i,'') 
    return x.strip()
    

def matchLocation(x:str) -> str:
    if x in locations_dict: # exact match
        location = locations_dict[x] 
        return location
    elif x in ['portugal', 'remote']:
        return x
    else:
        jarowinkler = JaroWinkler()
        loc, sim = max([(locations_dict[loc], jarowinkler.similarity(x, loc)) for loc in locations_dict], key=lambda x: x[1])
        if sim >= 0.6:
            return loc
        else: # no similarity match
            return f"NOT FOUND - {x} , {loc}, {sim}"
        

cleanJobLocation = compose(commonLocationFillers, replaceWithEmpty, matchLocation)

# How to use:
# applyFuncToColumn(test, function=cleanJobTitle, columns_list=['job_title'])

In [256]:
clean_test = (test.
              pipe(applyFuncToColumn, function=normalizeLocation, columns_list=["job_location"]).
              pipe(listToRows, column="job_location").
              pipe(applyFuncToColumn, function=cleanJobLocation, columns_list=["job_location"]).
              pipe(removeDupes, ['job_title', 'job_description','company', 'job_location']))

print(f'Previous shape: {test.shape}\nCurrent shape:{clean_test.shape}')

Previous shape: (109480, 10)
Current shape:(108193, 10)


In [260]:
totalJobsByYearMonth(df_clean)

job_title
post_year post_month           
2021      11              46458
          10              32370
          9                4018
          8                2072
          7                2249
          6                2788
          5                2947
          4                2427
          3                2353
          2                1742
          1                2100
2020      12               2261
          11               3485
          10                365
          9                 172
          8                 192
          7                 190
          6                 158
          5                 156
          4                 115
          3                 210
          2                 288
          1                 348
2018      11                  1

In [261]:
for i in clean_test.job_location.unique():
    print(i)

lisboa
setúbal
porto
ilha de são miguel
viana do castelo
braga
aveiro
ilha terceira
guarda
santarém
coimbra
leiria
bragança
beja
portugal
viseu
faro
castelo branco
ilha de porto santo
portalegre
remote
ilha da madeira
vila real
ilha de são jorge
ilha do faial
évora
NOT FOUND -  , aveiro, 0.0
ilha das flores
ilha do pico
ilha graciosa


In [259]:
print(matchLocation('Lisboa'))
print(matchLocation('benfica'))
print(matchLocation('bragança (portugal)'))
print(matchLocation('porto e etc'))
print(matchLocation('leiria as'))
print(matchLocation('aveiro district'))

lisboa
lisboa
bragança
porto
leiria
aveiro
